In [1]:
# ─── Colab Bootstrap (updated) ─────────────────────────────────────────────
import sys, os

if 'google.colab' in sys.modules:
    # 1. Download your repo as a ZIP
    !wget -q https://github.com/bobyellow/FlowLISA/archive/refs/heads/main.zip -O FlowLISA.zip
    # 2. Unzip it
    !unzip -q FlowLISA.zip
    # 3. Change into the extracted folder
    %cd FlowLISA-main

# 4. Ensure Python sees local modules
sys.path.insert(0, '.')

# Sanity check
print("Working dir:", os.getcwd())
print("Contents:", os.listdir('.'))
print("core/ present?", os.path.isdir('core'))


/content/FlowLISA-main
Working dir: /content/FlowLISA-main
Contents: ['core', '__pycache__', 'result', 'LICENSE', 'input', 'FlowLISA_main.py', 'README.md', 'FlowLISA_main.ipynb']
core/ present? True


In [8]:
import geopandas as gpd
from libpysal.weights import Queen
import pandas as pd

shp_path = '/content/FlowLISA-main/input/capital.shp'
csv_path = '/content/FlowLISA-main/input/converted_flow.txt'   # flow 데이터의 경로로 수정!

gdf = gpd.read_file(shp_path)
df_flow = pd.read_csv(csv_path, encoding='utf-8')



In [17]:
import geopandas as gpd
import pandas as pd

# 1. shp 파일에서 Polygon 순서(index) <-> ADM_CD_10 코드 mapping
gdf = gpd.read_file('/content/FlowLISA-main/input/capital.shp')
code_list = gdf['ADM_CD_10'].tolist()  # Polygon 인덱스와 1:1

code_to_index = {code: idx for idx, code in enumerate(code_list)}

# 2. flow 파일 로딩 (converted_flow.txt)
df_flow = pd.read_csv('/content/FlowLISA-main/input/converted_flow.txt', sep='\t')  # 또는 sep=',', encoding='utf-8'

# 3. (O, D) 코드 → (O_index, D_index) 변환
df_flow['O_idx'] = df_flow['O'].map(code_to_index)
df_flow['D_idx'] = df_flow['D'].map(code_to_index)
df_flow = df_flow.rename(columns={'Flow': 'V'})

# 4. "shp에 없는 코드" (매칭 불가) 제거
df_flow_valid = df_flow.dropna(subset=['O_idx', 'D_idx']).copy()

# 5. flow_dict 생성

# KeyError: 'V'가 절대 발생하지 않습니다!
flow_dict = {
    (int(row['O_idx']), int(row['D_idx'])): [row['V']]
    for _, row in df_flow_valid.iterrows() if row['V'] > 0
}


In [18]:
# 1. 인접성 분석으로 이웃 없는 섬 지역 찾기 (index가 아니라 ADM_CD_10 기준!)
w = Queen.from_dataframe(gdf)
# w.neighbors의 key는 gdf의 0-based index이므로,
# 이 index를 ADM_CD_10 코드로 mapping 해야 함
index_to_code = gdf['ADM_CD_10'].reset_index(drop=True)
islands_idx = [i for i, neighbors in w.neighbors.items() if len(neighbors) == 0]
islands_code = set(index_to_code.iloc[islands_idx])

print('이웃 없는 섬 지역 코드:', islands_code)

# 2. 섬 지역 제외 (gdf, flow 모두)
gdf_noisland = gdf[~gdf['ADM_CD_10'].isin(islands_code)].copy()
df_flow_noisland = df_flow[~df_flow['O'].isin(islands_code) & ~df_flow['D'].isin(islands_code)].copy()

# 3. AREAS 생성 (Polygon 좌표 리스트)
from shapely.geometry import Polygon, MultiPolygon
def to_pyshp_coords(geom):
    if isinstance(geom, Polygon):
        return [list(geom.exterior.coords)]
    elif isinstance(geom, MultiPolygon):
        return [list(poly.exterior.coords) for poly in geom.geoms]
    else:
        raise ValueError("Unsupported geometry type:", type(geom))

AREAS = [to_pyshp_coords(geom) for geom in gdf.geometry]


# 4. flow_dict 생성 (O, D는 코드)
flow_dict = {(row['O'], row['D']): [row['V']] for _, row in df_flow_noisland.iterrows() if row['V'] > 0}


/tmp/ipython-input-18-2758837852.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf)
/usr/local/lib/python3.11/dist-packages/libpysal/weights/contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 14 disconnected components.
 There are 11 islands with ids: 367, 560, 561, 562, 563, 564, 567, 568, 608, 609, 610.
  W.__init__(self, neighbors, ids=ids, **kw)


이웃 없는 섬 지역 코드: {2872036000.0, 2871040000.0, 4127361000.0, 2872033000.0, 2872037000.0, 2871041000.0, 2872035000.0, 2872038000.0, 2872034000.0, 2871042000.0, 2872031000.0}


In [2]:
import os
import core.shapefile
import pandas as pd
from core.FlowLISA import execFLOWLISA

# 1. 경로 설정 (Colab 업로드된 경로로)
shp_path = '/mnt/data/capital.shp'
flow_path = '/mnt/data/converted_flow.txt'

# 2. shp 파일에서 polygon 순서(index) ↔ ADM_CD_10 코드 mapping
StationPolygon = core.shapefile.Reader(shp_path)
shapes = StationPolygon.shapes()
records = StationPolygon.records()  # 속성값 접근

# 속성 테이블에서 ADM_CD_10이 몇번째 컬럼인지 확인
fields = StationPolygon.fields[1:]  # 첫번째 필드는 삭제해야함
field_names = [f[0] for f in fields]
adm_idx = field_names.index('ADM_CD_10')
code_list = [r[adm_idx] for r in records]  # Polygon 인덱스 → 코드
code_to_index = {code: idx for idx, code in enumerate(code_list)}

# 3. flow 데이터 읽고, (O, D)코드를 (O_index, D_index)로 변환
df_flow = pd.read_csv(flow_path, sep='\t')  # 컬럼: O, D, Flow

df_flow['O_idx'] = df_flow['O'].map(code_to_index)
df_flow['D_idx'] = df_flow['D'].map(code_to_index)

df_flow_valid = df_flow.dropna(subset=['O_idx', 'D_idx']).copy()
df_flow_valid = df_flow_valid[df_flow_valid['Flow'] > 0]  # 0 flow 제거

# 4. flow dict: (O_index, D_index): [Flow]
F_dt1 = {
    (int(row['O_idx']), int(row['D_idx'])): [row['Flow']]
    for _, row in df_flow_valid.iterrows()
}

# 5. AREAS 구조는 예제와 동일하게 polygon points 리스트
AREAS = [[shape.points] for shape in shapes]

# 6. FlowLISA 실행
outputStr = execFLOWLISA(AREAS, AREAS, F_dt1, 1, 120)

# 7. 결과 저장
output_dir = '/mnt/data/result'
os.makedirs(output_dir, exist_ok=True)
output_filename = os.path.join(output_dir, 'FlowLISA_I_result.txt')
with open(output_filename, 'w') as outputFile:
    outputFile.write(outputStr)

print(f"Processing complete. Results saved to {output_filename}")


Obtain O & D neighbors by contiguity (Rook or Queen)


IndexError: list index out of range